In [2]:
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None
exp_path = Path.cwd().resolve() # experiment path
# visible GPUs
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [15]:
import numpy as np
import pandas as pd
import cv2 as cv
from PIL import Image

The idea of the bridge is to generate high quality Mammo2CEm images training our pipeline using patches of the TRAINING images.<br>
The patches will be 512x512.

In [123]:
train_metadata_path = repo_path / 'data/CDD-CESM/metadata/bboxes/split_1/train_set.csv'
im_dir_CEM = repo_path / 'data/CDD-CESM/images/substracted'
im_dir_low = repo_path / 'data/CDD-CESM/images/low-energy'
train_metadata = pd.read_csv(train_metadata_path)

# saving dir
save_dir = repo_path / 'generation/bbridge/data/CEM-512/split_1/train'
save_dir.mkdir(parents=True, exist_ok=True)

In [133]:
ex_im_path = im_dir_CEM / (train_metadata['image_name'][20] + '.jpg')
low_im_path = im_dir_low / (ex_im_path.stem.replace('CM', 'DM') + '.jpg')


# read the image
ex_im = cv.imread(str(ex_im_path), cv.IMREAD_GRAYSCALE)
low_im = cv.imread(str(low_im_path), cv.IMREAD_GRAYSCALE)
print(f'Original shape: {ex_im.shape}')


# create a subimage of 512x512, starting from the 0,0 corner
i_y = 0
i_x = 0
y_shift = 512*i_y
x_shift = 512*i_x

from_y = 0+y_shift
to_y = 512+y_shift
from_x = 0+x_shift
to_x = 512+x_shift

if to_y > ex_im.shape[0]: # when to_y is too large (end of image)
    to_complete_y = 512 - (ex_im.shape[0] - from_y)
    to_y = ex_im.shape[0]

    sub_im = np.zeros((512, 512), dtype=np.uint8)
    # fill the subimage with the original image
    sub_im[:to_y-from_y, :] = ex_im[from_y:to_y, from_x:to_x]
    # replicate for low energy image
    sub_low_im = np.zeros((512, 512), dtype=np.uint8)
    sub_low_im[:to_y-from_y, :] = low_im[from_y:to_y, from_x:to_x]
elif to_x > ex_im.shape[1]:
    to_complete_x = 512 - (ex_im.shape[1] - from_x)
    to_x = ex_im.shape[1]

    sub_im = np.zeros((512, 512), dtype=np.uint8)
    # fill the subimage with the original image
    sub_im[:, :to_x-from_x] = ex_im[from_y:to_y, from_x:to_x]
    # replicate for low energy image
    sub_low_im = np.zeros((512, 512), dtype=np.uint8)
    sub_low_im[:, :to_x-from_x] = low_im[from_y:to_y, from_x:to_x]

else: # when limits are within the image
    sub_im = ex_im[from_y:to_y, from_x:to_x]
    sub_low_im = low_im[from_y:to_y, from_x:to_x]

# if more than 30% of the image is black, then the image is considered black
black = np.sum(sub_im == 0) / (512*512) > 0.7
sub_im = Image.fromarray(sub_im)
sub_low_im = Image.fromarray(sub_low_im)
print(f'Image from y:{from_y} to y:{to_y} and x:{from_x} to x:{to_x} is black: {black}')
# save image in the save_dir
save_path = save_dir / 'B' / f'{ex_im_path.stem}_y{i_y}_x{i_x}.jpg'
save_path.parent.mkdir(parents=True, exist_ok=True)
sub_im.save(save_path)

save_path_low = save_dir / 'A' / f'{low_im_path.stem}_y{i_y}_x{i_x}.jpg'
save_path_low.parent.mkdir(parents=True, exist_ok=True)
sub_low_im.save(save_path_low)


Original shape: (2620, 1657)
Image from y:0 to y:512 and x:0 to x:512 is black: False
